```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNp1kE9rhDAQxb9KmJMLG8Grh57agyAVKpSC6SFrRs2uJjZOWmTZ796o_bulc0pmXn7vTc5QW4WQQtPbt7qTjhjLH4RhoSZ_aJ0cO5aZ0RPL5Yxum5QuQkOaZm7kgDvGORPQ6B6Xa1xPrwJC74aVVTSRbLVpd8_bSzTqD5zQDai0JPzlsRKyKtI_BP9jCk9XIbMVUCRV5I1-8cjtKuH9ItmCf9PYFU7AU6j4OFkjYJsVCePxsqdMG8mnURsTrIDFweXxPk-iRczt4Yg1bfgvOuxhCDtIrcJPn5e2AOpwQAFpOCrpTovLJej8qMKed0qTdRCM-gn3ID3ZcjY1pOQ8foputQxhhw_V5R2AMpSJ)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNp1kE9rhDAQxb9KmJMLG8Grh57agyAVKpSC6SFrRs2uJjZOWmTZ796o_bulc0pmXn7vTc5QW4WQQtPbt7qTjhjLH4RhoSZ_aJ0cO5aZ0RPL5Yxum5QuQkOaZm7kgDvGORPQ6B6Xa1xPrwJC74aVVTSRbLVpd8_bSzTqD5zQDai0JPzlsRKyKtI_BP9jCk9XIbMVUCRV5I1-8cjtKuH9ItmCf9PYFU7AU6j4OFkjYJsVCePxsqdMG8mnURsTrIDFweXxPk-iRczt4Yg1bfgvOuxhCDtIrcJPn5e2AOpwQAFpOCrpTovLJej8qMKed0qTdRCM-gn3ID3ZcjY1pOQ8foputQxhhw_V5R2AMpSJ)

`UPDATE DIAGRAM BY CLICKING ON THE DIAGRAM!`

Required steps:

1. `ADD REQUIRMENT STEP LIST HERE!`

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]'))
CREATE TABLE [VWSSTAGE].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] (
	[ID] [BIGINT] IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[VERSION] VARCHAR(50),
	[DATE_OF_REPORT] VARCHAR(50),
	[DATE_OF_STATISTICS] VARCHAR(50),
	[PERC_COVID_SYMPTOMS] VARCHAR(50),
	[MA_PERC_COVID_SYMPTOMS] VARCHAR(50),
	[AGE_GROUPS] VARCHAR(50)
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]'))
CREATE TABLE [VWSINTER].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] (
	[ID] [BIGINT] IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[VERSION] INT NULL,
	[DATE_OF_REPORT] DATETIME2 NULL,
	[DATE_OF_STATISTICS] DATETIME2 NULL,
	[PERC_COVID_SYMPTOMS] FLOAT NULL,
	[MA_PERC_COVID_SYMPTOMS] FLOAT NULL,
	[AGE_GROUPS] VARCHAR(50) NULL
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] (
        [VERSION],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [PERC_COVID_SYMPTOMS],
        [MA_PERC_COVID_SYMPTOMS],
        [AGE_GROUPS]
    )
    SELECT
        [VERSION],
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_STATISTICS]),
        CONVERT(FLOAT, IIF([PERC_COVID_SYMPTOMS] IN ('', 'NA'), NULL, Replace([PERC_COVID_SYMPTOMS], ',', '.'))),
        CONVERT(FLOAT, IIF([MA_PERC_COVID_SYMPTOMS] IN ('', 'NA'), NULL, Replace([MA_PERC_COVID_SYMPTOMS], ',', '.'))),
        [AGE_GROUPS]
    FROM
        [VWSSTAGE].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>INFECTIE RADAR TREND PER LEEFTIJD </span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]'))
CREATE TABLE [VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[VERSION] INT NULL,
	[DATE_OF_REPORT] DATETIME2 NULL,
	[DATE_OF_STATISTICS] DATETIME2 NULL,
	[PERC_COVID_SYMPTOMS] FLOAT NULL,
	[MA_PERC_COVID_SYMPTOMS] FLOAT NULL,
	[AGE_GROUPS] VARCHAR(50) NULL
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] 
	ON [VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]
    ON [VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]
AS
BEGIN
    INSERT INTO [VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] (
        [VERSION],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [PERC_COVID_SYMPTOMS],
        [MA_PERC_COVID_SYMPTOMS],
        [AGE_GROUPS]
    )
    SELECT
        [VERSION],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [PERC_COVID_SYMPTOMS],
        [MA_PERC_COVID_SYMPTOMS],
        [AGE_GROUPS]
    FROM
        [VWSINTER].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD])
END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSREPORT].[V_RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] AS
WITH CTE AS (
    SELECT
        [VERSION],
        [DATE_LAST_INSERTED],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [PERC_COVID_SYMPTOMS],
        [MA_PERC_COVID_SYMPTOMS],
        [AGE_GROUPS]
    FROM [VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_INFECTIE_RADAR_TREND_PER_LEEFTIJD] WITH(NOLOCK))
)
SELECT
    *
FROM CTE
GO